In [1]:
# Document to Vector

import gensim.models.keyedvectors as word2vec
import numpy as np

class DocumentToVector:

    def __init__(self):
        # Pre-Trained Word2Vec data
        filename = 'GoogleNews-vectors-negative300.bin.gz'
        self.__word2vec_model = word2vec.KeyedVectors.load_word2vec_format(filename, binary=True)
        # Normalize vectors
        self.__word2vec_model.init_sims(replace=True)

    def getWord2Vec(self, word):
        # returns word2vec representation of a word
        return self.__word2vec_model[word]

    def getDoc2Vec(self, doc):
        # returns the doc2vec representation of a doc: doc2vec is average of all word2vec in a document
        doc = [word for word in doc if word in self.__word2vec_model.vocab]
        # Check if at least one word in the vocab is present in the doc
        if(len(doc)):
            return np.mean(self.__word2vec_model[doc], axis=0)
        else:
            return np.array([0])

In [2]:
doc2vec = DocumentToVector()

# Check Word2Vec for one word

In [9]:
# Stem Word
doc2vec.getWord2Vec('univer')

array([-0.06127518, -0.00046777,  0.01531879,  0.11064546,  0.04831985,
        0.02958716, -0.05007057, -0.0885864 ,  0.02232167, -0.05042072,
        0.04219234, -0.07633136, -0.04026654,  0.06617719,  0.01689444,
        0.08123337,  0.0056023 ,  0.00067567,  0.03168802,  0.09874057,
       -0.07773194, -0.11064546,  0.05182129,  0.01207996,  0.03746539,
       -0.01549387, -0.07388035,  0.00748432,  0.051121  , -0.05147115,
        0.01418083,  0.01829502,  0.01199243, -0.02468514,  0.02004573,
       -0.07948265, -0.04586884,  0.00932258,  0.07142935,  0.01470604,
        0.08473481,  0.00976026, -0.02205906,  0.05357201,  0.12185007,
       -0.03694018, -0.00770317,  0.02783644, -0.09453885,  0.00114344,
        0.0399164 , -0.04867   , -0.07282992,  0.05252158,  0.04044162,
        0.10014115, -0.04254248, -0.03466424, -0.0255605 , -0.03168802,
       -0.04586884, -0.09313826, -0.00577737, -0.07878236,  0.0221466 ,
       -0.03834075, -0.00608375,  0.03974133, -0.04831985,  0.00

# Check Doc2Vec for one doc

In [15]:
doc = '''
Markov chain Monte Carlo (MCMC) methods are an important algorithmic
 device in a variety of fields.  This project studies techniques for rigorous
 analysis of the convergence properties of Markov chains.   The emphasis is on
 refining probabilistic, analytic and combinatorial tools (such as coupling,
 log-Sobolev, and canonical paths) to improve existing algorithms and develop
 efficient algorithms for important open problems.
 Problems arising in
 computer science, discrete mathematics, and physics are of particular interest,
 e.g., generating random colorings and independent sets of bounded-degree
 graphs, approximating the permanent, estimating the volume of a convex body,
 and sampling contingency tables.  The project also studies inherent connections
 between phase transitions in statistical physics models and convergence
 properties of associated Markov chains.
 The investigator is developing a
 new graduate course on MCMC methods.
'''

doc_vec = doc2vec.getDoc2Vec(doc)
print('Single data point shape: ', doc_vec.shape)
print(doc_vec)

Single data point shape:  (300,)
[-0.06829835  0.04124255 -0.00076544  0.05486702 -0.02108179  0.00702612
 -0.03165184 -0.01618573 -0.01916436  0.00832311 -0.02320279 -0.03042367
 -0.07769621  0.01089161 -0.04650958  0.03417463  0.03414661  0.06438107
 -0.00815716  0.00247857 -0.09712694 -0.00799085  0.04122848  0.00766652
 -0.03431297  0.01532645 -0.09797373  0.02784445 -0.01405189 -0.00652317
  0.00282243  0.00953653 -0.02713961 -0.04282871 -0.05239114  0.03940746
 -0.08098549  0.04569623 -0.02462404  0.03219333 -0.01812342 -0.02287296
  0.023243    0.0410194   0.0174388  -0.00379472 -0.015629   -0.06659897
 -0.0390147   0.0356218  -0.06269674  0.09066872 -0.01025739  0.09139203
  0.01470939  0.04962143 -0.0611753  -0.03351038 -0.00144735 -0.05910344
 -0.05397749 -0.02539386 -0.07083242 -0.01976186 -0.01421504 -0.07324556
 -0.03390598  0.04734793 -0.02636825  0.01391274  0.01177966 -0.03179437
  0.01336408 -0.00788371 -0.01069529 -0.00388562  0.05108869  0.00429456
  0.00948141 -0.03

# Pre-Process all the files in directory tree

In [4]:
class AbstractExtrator:

    def __init__(self):
        pass

    def extractAbstract(self, filepath):
        # Extracts abstract content from given file
        line_index = 0
        with open(filepath, encoding='iso-8859-1') as f:  
        #with open(filepath) as f:  
            content = f.readlines()
            for line in content:
                #print(line)
                line_index += 1
                # Extract unique document identifier: 'Award Number'
                if line.find('Award Number:') != -1:
                    awardNo = line.split(':')[1].strip()
                if line.find('Abstract    :') != -1:
                    break

        #print('line_index:', line_index)
        # Combine all lines after 'Abstract    :' key
        abstract = ' '.join(content[line_index:]).strip()
        return abstract, awardNo
    
    def display_doc(self, filepath):
        with open(filepath, encoding='iso-8859-1') as f:  
            content = f.readlines()
            for line in content:
                print(line)
    

In [5]:
def docs_filter_(doc):
    if (len(doc) == 0):
        return None
    
    # Some documents contain NotAvailable keyword for empty abstracts which needs to be filtered
    NotAvailable = "Not Available"
    # Also contain some spaces after making total of 28 words
    NotAvailable_ContentLen = 28  # Found by looking at raw data
    if (doc.find(NotAvailable) != -1 and len(doc) == NotAvailable_ContentLen):
        #print(NotAvailable + " found")
        #print(len(doc))
        return None
    
    return doc

In [11]:
# Preprocess every file in the directory
import os
from fnmatch import fnmatch
pattern = "*.txt"

def list_files(dir):
    r = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            if fnmatch(name, pattern):
                r.append(os.path.join(root, name))
    return r

files = list_files('./Part1/')
#print(files))

docs = []
docs_vector = []

no_valid_files = 0
no_skipped_files = 0
for file in files:
    print(file)
    
    # Extract Abstract
    absExt = AbstractExtrator()
    doc, award = absExt.extractAbstract(file)
    #print('Award: ', award)
    if award == 'null':
        print('\t\t\t\t\t\t Skipped due to null Award')
        no_skipped_files += 1
        continue
    
    # Skipping Blank Abstract
    if(docs_filter_(doc) == None):
        print('\t\t\t\t\t\t Skipped due to blank Abstract')
        no_skipped_files += 1
        continue
    
    docs.append(doc)
    # Determine vector representation of the document
    vec = doc2vec.getDoc2Vec(doc)
    
    if(len(vec) != 1):
        vec = np.insert(vec, 0, award)
        #print(vec)
        docs_vector.append(vec)
        no_valid_files += 1
        
    else:
        print('\t\t\t\t\t\t Skipped due to poor vocab')
        no_skipped_files += 1
        continue
    
    #break
    
print('No of valid files = ', no_valid_files)
print('No of skipped files scanned = ', no_skipped_files)

./Part1/awards_1990/awd_1990_00/a9000006.txt
./Part1/awards_1990/awd_1990_00/a9000031.txt
./Part1/awards_1990/awd_1990_00/a9000038.txt
./Part1/awards_1990/awd_1990_00/a9000040.txt
./Part1/awards_1990/awd_1990_00/a9000043.txt
./Part1/awards_1990/awd_1990_00/a9000045.txt
./Part1/awards_1990/awd_1990_00/a9000046.txt
./Part1/awards_1990/awd_1990_00/a9000048.txt
./Part1/awards_1990/awd_1990_00/a9000049.txt
./Part1/awards_1990/awd_1990_00/a9000050.txt
./Part1/awards_1990/awd_1990_00/a9000052.txt
./Part1/awards_1990/awd_1990_00/a9000053.txt
./Part1/awards_1990/awd_1990_00/a9000054.txt
./Part1/awards_1990/awd_1990_00/a9000057.txt
./Part1/awards_1990/awd_1990_00/a9000058.txt
./Part1/awards_1990/awd_1990_00/a9000060.txt
./Part1/awards_1990/awd_1990_00/a9000063.txt
./Part1/awards_1990/awd_1990_00/a9000075.txt
./Part1/awards_1990/awd_1990_00/a9000089.txt
./Part1/awards_1990/awd_1990_00/a9000091.txt
./Part1/awards_1990/awd_1990_00/a9000094.txt
./Part1/awards_1990/awd_1990_00/a9000099.txt
./Part1/aw

./Part1/awards_1990/awd_1990_00/a9000644.txt
./Part1/awards_1990/awd_1990_00/a9000645.txt
./Part1/awards_1990/awd_1990_00/a9000647.txt
./Part1/awards_1990/awd_1990_00/a9000648.txt
./Part1/awards_1990/awd_1990_00/a9000650.txt
./Part1/awards_1990/awd_1990_00/a9000651.txt
./Part1/awards_1990/awd_1990_00/a9000653.txt
./Part1/awards_1990/awd_1990_00/a9000654.txt
./Part1/awards_1990/awd_1990_00/a9000663.txt
./Part1/awards_1990/awd_1990_00/a9000665.txt
./Part1/awards_1990/awd_1990_00/a9000666.txt
./Part1/awards_1990/awd_1990_00/a9000670.txt
./Part1/awards_1990/awd_1990_00/a9000671.txt
./Part1/awards_1990/awd_1990_00/a9000675.txt
./Part1/awards_1990/awd_1990_00/a9000676.txt
./Part1/awards_1990/awd_1990_00/a9000678.txt
./Part1/awards_1990/awd_1990_00/a9000681.txt
./Part1/awards_1990/awd_1990_00/a9000683.txt
./Part1/awards_1990/awd_1990_00/a9000687.txt
./Part1/awards_1990/awd_1990_00/a9000688.txt
./Part1/awards_1990/awd_1990_00/a9000689.txt
./Part1/awards_1990/awd_1990_00/a9000692.txt
./Part1/aw

./Part1/awards_1990/awd_1990_01/a9001096.txt
./Part1/awards_1990/awd_1990_01/a9001103.txt
./Part1/awards_1990/awd_1990_01/a9001104.txt
./Part1/awards_1990/awd_1990_01/a9001106.txt
./Part1/awards_1990/awd_1990_01/a9001107.txt
./Part1/awards_1990/awd_1990_01/a9001108.txt
./Part1/awards_1990/awd_1990_01/a9001115.txt
./Part1/awards_1990/awd_1990_01/a9001116.txt
./Part1/awards_1990/awd_1990_01/a9001120.txt
./Part1/awards_1990/awd_1990_01/a9001126.txt
./Part1/awards_1990/awd_1990_01/a9001127.txt
./Part1/awards_1990/awd_1990_01/a9001134.txt
./Part1/awards_1990/awd_1990_01/a9001137.txt
./Part1/awards_1990/awd_1990_01/a9001139.txt
./Part1/awards_1990/awd_1990_01/a9001146.txt
./Part1/awards_1990/awd_1990_01/a9001152.txt
./Part1/awards_1990/awd_1990_01/a9001154.txt
./Part1/awards_1990/awd_1990_01/a9001156.txt
./Part1/awards_1990/awd_1990_01/a9001157.txt
./Part1/awards_1990/awd_1990_01/a9001164.txt
./Part1/awards_1990/awd_1990_01/a9001169.txt
./Part1/awards_1990/awd_1990_01/a9001170.txt
./Part1/aw

./Part1/awards_1990/awd_1990_01/a9001689.txt
./Part1/awards_1990/awd_1990_01/a9001690.txt
./Part1/awards_1990/awd_1990_01/a9001691.txt
./Part1/awards_1990/awd_1990_01/a9001693.txt
./Part1/awards_1990/awd_1990_01/a9001695.txt
./Part1/awards_1990/awd_1990_01/a9001696.txt
./Part1/awards_1990/awd_1990_01/a9001697.txt
./Part1/awards_1990/awd_1990_01/a9001698.txt
./Part1/awards_1990/awd_1990_01/a9001699.txt
./Part1/awards_1990/awd_1990_01/a9001701.txt
./Part1/awards_1990/awd_1990_01/a9001702.txt
./Part1/awards_1990/awd_1990_01/a9001704.txt
./Part1/awards_1990/awd_1990_01/a9001705.txt
./Part1/awards_1990/awd_1990_01/a9001707.txt
./Part1/awards_1990/awd_1990_01/a9001710.txt
./Part1/awards_1990/awd_1990_01/a9001712.txt
./Part1/awards_1990/awd_1990_01/a9001714.txt
./Part1/awards_1990/awd_1990_01/a9001717.txt
./Part1/awards_1990/awd_1990_01/a9001721.txt
./Part1/awards_1990/awd_1990_01/a9001722.txt
./Part1/awards_1990/awd_1990_01/a9001726.txt
./Part1/awards_1990/awd_1990_01/a9001729.txt
./Part1/aw

./Part1/awards_1990/awd_1990_02/a9002279.txt
./Part1/awards_1990/awd_1990_02/a9002286.txt
./Part1/awards_1990/awd_1990_02/a9002289.txt
./Part1/awards_1990/awd_1990_02/a9002292.txt
./Part1/awards_1990/awd_1990_02/a9002294.txt
./Part1/awards_1990/awd_1990_02/a9002299.txt
./Part1/awards_1990/awd_1990_02/a9002300.txt
./Part1/awards_1990/awd_1990_02/a9002315.txt
./Part1/awards_1990/awd_1990_02/a9002318.txt
./Part1/awards_1990/awd_1990_02/a9002321.txt
./Part1/awards_1990/awd_1990_02/a9002326.txt
./Part1/awards_1990/awd_1990_02/a9002330.txt
./Part1/awards_1990/awd_1990_02/a9002331.txt
./Part1/awards_1990/awd_1990_02/a9002332.txt
./Part1/awards_1990/awd_1990_02/a9002333.txt
./Part1/awards_1990/awd_1990_02/a9002335.txt
./Part1/awards_1990/awd_1990_02/a9002341.txt
./Part1/awards_1990/awd_1990_02/a9002342.txt
./Part1/awards_1990/awd_1990_02/a9002345.txt
./Part1/awards_1990/awd_1990_02/a9002347.txt
./Part1/awards_1990/awd_1990_02/a9002349.txt
./Part1/awards_1990/awd_1990_02/a9002351.txt
./Part1/aw

...

./Part1/awards_1994/awd_1994_54/a9454638.txt
./Part1/awards_1994/awd_1994_54/a9454640.txt
./Part1/awards_1994/awd_1994_54/a9454642.txt
./Part1/awards_1994/awd_1994_54/a9454643.txt
./Part1/awards_1994/awd_1994_54/a9454647.txt
./Part1/awards_1994/awd_1994_54/a9454648.txt
./Part1/awards_1994/awd_1994_54/a9454651.txt
./Part1/awards_1994/awd_1994_54/a9454654.txt
./Part1/awards_1994/awd_1994_54/a9454655.txt
./Part1/awards_1994/awd_1994_54/a9454657.txt
./Part1/awards_1994/awd_1994_54/a9454660.txt
./Part1/awards_1994/awd_1994_54/a9454691.txt
./Part1/awards_1994/awd_1994_54/a9454703.txt
./Part1/awards_1994/awd_1994_54/a9454704.txt
./Part1/awards_1994/awd_1994_54/a9454706.txt
./Part1/awards_1994/awd_1994_54/a9454707.txt
./Part1/awards_1994/awd_1994_54/a9454714.txt
./Part1/awards_1994/awd_1994_54/a9454715.txt
./Part1/awards_1994/awd_1994_54/a9454719.txt
./Part1/awards_1994/awd_1994_54/a9454729.txt
./Part1/awards_1994/awd_1994_54/a9454732.txt
./Part1/awards_1994/awd_1994_54/a9454741.txt
./Part1/aw

./Part1/awards_1994/awd_1994_57/a9457039.txt
./Part1/awards_1994/awd_1994_57/a9457049.txt
./Part1/awards_1994/awd_1994_57/a9457083.txt
./Part1/awards_1994/awd_1994_57/a9457097.txt
./Part1/awards_1994/awd_1994_57/a9457108.txt
./Part1/awards_1994/awd_1994_57/a9457111.txt
./Part1/awards_1994/awd_1994_57/a9457126.txt
./Part1/awards_1994/awd_1994_57/a9457133.txt
./Part1/awards_1994/awd_1994_57/a9457148.txt
./Part1/awards_1994/awd_1994_57/a9457154.txt
./Part1/awards_1994/awd_1994_57/a9457156.txt
./Part1/awards_1994/awd_1994_57/a9457168.txt
./Part1/awards_1994/awd_1994_57/a9457179.txt
./Part1/awards_1994/awd_1994_57/a9457189.txt
./Part1/awards_1994/awd_1994_57/a9457205.txt
./Part1/awards_1994/awd_1994_57/a9457206.txt
./Part1/awards_1994/awd_1994_57/a9457212.txt
./Part1/awards_1994/awd_1994_57/a9457217.txt
./Part1/awards_1994/awd_1994_57/a9457236.txt
./Part1/awards_1994/awd_1994_57/a9457245.txt
./Part1/awards_1994/awd_1994_57/a9457246.txt
./Part1/awards_1994/awd_1994_57/a9457259.txt
./Part1/aw

./Part1/awards_1994/awd_1994_58/a9458356.txt
./Part1/awards_1994/awd_1994_58/a9458367.txt
./Part1/awards_1994/awd_1994_58/a9458368.txt
./Part1/awards_1994/awd_1994_58/a9458373.txt
./Part1/awards_1994/awd_1994_58/a9458376.txt
./Part1/awards_1994/awd_1994_58/a9458405.txt
./Part1/awards_1994/awd_1994_58/a9458418.txt
./Part1/awards_1994/awd_1994_58/a9458424.txt
./Part1/awards_1994/awd_1994_58/a9458427.txt
./Part1/awards_1994/awd_1994_60/a9460011.txt
./Part1/awards_1994/awd_1994_60/a9460015.txt
./Part1/awards_1994/awd_1994_60/a9460027.txt
./Part1/awards_1994/awd_1994_60/a9460038.txt
./Part1/awards_1994/awd_1994_60/a9460043.txt
./Part1/awards_1994/awd_1994_60/a9460046.txt
./Part1/awards_1994/awd_1994_60/a9460057.txt
./Part1/awards_1994/awd_1994_60/a9460058.txt
./Part1/awards_1994/awd_1994_60/a9460060.txt
./Part1/awards_1994/awd_1994_60/a9460066.txt
./Part1/awards_1994/awd_1994_60/a9460067.txt
./Part1/awards_1994/awd_1994_60/a9460072.txt
./Part1/awards_1994/awd_1994_60/a9460073.txt
./Part1/aw

./Part1/awards_1994/awd_1994_61/a9461370.txt
./Part1/awards_1994/awd_1994_61/a9461386.txt
./Part1/awards_1994/awd_1994_61/a9461389.txt
./Part1/awards_1994/awd_1994_61/a9461400.txt
./Part1/awards_1994/awd_1994_61/a9461414.txt
./Part1/awards_1994/awd_1994_61/a9461421.txt
./Part1/awards_1994/awd_1994_61/a9461435.txt
./Part1/awards_1994/awd_1994_61/a9461439.txt
./Part1/awards_1994/awd_1994_61/a9461450.txt
./Part1/awards_1994/awd_1994_61/a9461452.txt
./Part1/awards_1994/awd_1994_61/a9461460.txt
./Part1/awards_1994/awd_1994_61/a9461464.txt
./Part1/awards_1994/awd_1994_61/a9461465.txt
./Part1/awards_1994/awd_1994_61/a9461466.txt
./Part1/awards_1994/awd_1994_61/a9461472.txt
./Part1/awards_1994/awd_1994_61/a9461476.txt
./Part1/awards_1994/awd_1994_61/a9461485.txt
./Part1/awards_1994/awd_1994_61/a9461490.txt
./Part1/awards_1994/awd_1994_61/a9461493.txt
./Part1/awards_1994/awd_1994_61/a9461494.txt
./Part1/awards_1994/awd_1994_61/a9461499.txt
./Part1/awards_1994/awd_1994_61/a9461500.txt
./Part1/aw

./Part1/awards_1994/awd_1994_96/a9496170.txt
./Part1/awards_1994/awd_1994_96/a9496171.txt
./Part1/awards_1994/awd_1994_96/a9496172.txt
./Part1/awards_1994/awd_1994_96/a9496173.txt
./Part1/awards_1994/awd_1994_96/a9496174.txt
./Part1/awards_1994/awd_1994_96/a9496175.txt
./Part1/awards_1994/awd_1994_96/a9496176.txt
./Part1/awards_1994/awd_1994_96/a9496177.txt
./Part1/awards_1994/awd_1994_96/a9496178.txt
./Part1/awards_1994/awd_1994_96/a9496179.txt
./Part1/awards_1994/awd_1994_96/a9496180.txt
./Part1/awards_1994/awd_1994_96/a9496181.txt
./Part1/awards_1994/awd_1994_96/a9496182.txt
./Part1/awards_1994/awd_1994_96/a9496183.txt
./Part1/awards_1994/awd_1994_96/a9496184.txt
./Part1/awards_1994/awd_1994_96/a9496185.txt
./Part1/awards_1994/awd_1994_96/a9496186.txt
./Part1/awards_1994/awd_1994_96/a9496187.txt
./Part1/awards_1994/awd_1994_96/a9496188.txt
./Part1/awards_1994/awd_1994_96/a9496189.txt
./Part1/awards_1994/awd_1994_96/a9496191.txt
./Part1/awards_1994/awd_1994_96/a9496192.txt
./Part1/aw

# Write the document vectors into files for easy post-processing

In [8]:
# Write docs vec to an output file
import  csv
with open("docs_vector.csv","w") as f:
    wr = csv.writer(f)
    wr.writerows(docs_vector)

# Testing Purpose Only

In [9]:
# # Testing if docs are extracted properly
# import  csv
# with open("docs.txt","w") as f:
#     for doc in docs:
#         if(docs_filter_(doc) != None):
#             f.write(doc)
#             f.write("***********************************************************\r\n")        

In [12]:
# Testing random abstracts
absExt1 = AbstractExtrator()
test_abstract, test_awardNo = absExt1.extractAbstract('./Part1/awards_1991/awd_1991_13/a9113226.txt')
print('Test Abstract: ', test_abstract)
print('Test awardNo: ', test_awardNo)

Test Abstract:  Abstractions are the basis for most of the ways that programmers look          
               at their programs, data, and execution.  Most of these abstractions            
               have a visual component.  This work envisions a system that will allow         
               programmers to rapidly define abstractions and their accompanying              
               visualizations.  Such a system would have immediate applications to            
               programming environments, data bases, parallel programming, and                
               education.  This project will investigate what it means to design such         
               a system and to explore many of the underlying problems that must be           
               solved before such a system can become a reality.  In conjunction with         
               other research projects, it will lead to an abstraction-based                  
               visualization environment to exper